In [ ]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
#import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
!pip install transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.7 MB/s eta 0:00:00


<ipython-input-2-335695ed4efd>:45: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


<Figure size 432x288 with 0 Axes>

In [ ]:
df=pd.read_csv("Disaster_dataset.csv")

In [ ]:
def remove_stopwords(sentence):
    """
    Removes a list of stopwords

    Args:
        sentence (string): sentence to remove the stopwords from

    Returns:
        sentence (string): lowercase sentence without the stopwords
    """
    # List of stopwords
    stopwords = ["a", "about", "above", "after", "again", "ago","against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

    # Sentence converted to lowercase-only
    sentence = str(sentence).lower()

    words = sentence.split()
    no_words = [w for w in words if w not in stopwords]
    sentence = " ".join(no_words)

    return sentence

df['Tweet']=df['Tweet'].apply(lambda x:remove_stopwords(x))




In [ ]:

import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet from NLTK
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmi(sentence):


    # Tokenize the sentence into individual words
    words = nltk.word_tokenize(sentence)

    # Lemmatize each word to its base form
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = " ".join(lemmatized_words)

    # Print the original and lemmatized sentences

    return  lemmatized_sentence
df['Tweet']=df['Tweet'].apply(lambda x:lemmi(x))
df["Tweet"]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0       earthquake preparedness never stop young stude...
1       uae two aid plane carrying 553 ton relief item...
2       football fan besiktas throw toy pitch honour t...
3       usgs report m085 earthquake 3km w anderson spr...
4       haytap turkey gt small miracle 21st day turkey...
                              ...                        
5540    _globalcrisis_ destruction colossal going get ...
5541    climatecerberus damage still chance must use u...
5542    _globalcrisis_ climatecerberus almost off chai...
5543    chaosdujour comey fbi spaghettimodel senstivet...
5544    warning florida will suffer direct hit categor...
Name: Tweet, Length: 5545, dtype: object

In [ ]:
MAX_LEN=128

In [ ]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
def create_model(bert_model, max_len=MAX_LEN):

    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    embeddings = bert_model([input_ids,attention_masks])[1]

    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

    model.compile(opt, loss=loss, metrics=accuracy)


    return model

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
token_lens = []

for txt in df['Tweet'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

max_len=np.max(token_lens)
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 57


In [ ]:
token_lens = []

for i,txt in enumerate(df['Tweet'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")


In [ ]:
df['token_lens'] = token_lens

In [ ]:
df = df.sort_values(by='token_lens', ascending=False)
df.head(20)

,Tweet,Label,token_lens
5121,dyhyh5v6vz mzedwards40 stefanosstantz jmtech20...,Other disaster,57
4718,dmkalahandi dfosouth forestdeptt pccfodisha su...,Other disaster,47
4719,dmkalahandi dfosouth forestdeptt pccfodisha su...,Other disaster,45
3907,roadaccident uppolice uptrafficpolice myogiadi...,Accidnet,44
5303,jeromerovolcan uom_ees marghepolacci mikeburto...,Other disaster,43
3623,prashant221278 dr_ashutosh_md samarth_6 5waves...,Accidnet,43
1509,soniacolpo sari_friends1 mrtom101 sharuana hr1...,Earthquake,42
2397,flooded roadway south london ekywx kywx spann ...,Flood,40
1605,want sell wts wtb aab pc nct nct 127 gt goodco...,Earthquake,40
371,6months used 2015toyota corolla lefirst body b...,Earthquake,39


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# df['Label'] = df['Label'].map({'Eartquake':0,'Flood':1,'Accident':2,'Other Disaster':3})

In [ ]:
#In this step we are Encoding Our target variable which is Theme as we can see numbers from 0-6 have been assinged to every unique label
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Label=df['Label']
Label=np.array(Label).reshape(-1,1)


df['encoded_labels']=''
enc=le.fit_transform(Label)

df['encoded_labels']=enc
df.head()

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Tweet,Label,token_lens,encoded_labels
0,volcanoologists expert study volcano,Other disaster,7,3
1,longest english word pneumonoultramicroscopics...,Other disaster,29,3
2,1996 1 year t photo del lagrace volcano 2023 2...,Other disaster,25,3
3,wu lit ricchdawg01 law legaltok lawyersoftikto...,Accidnet,32,0
4,upcoming conference berlin advertises surrogac...,Flood,22,2


In [ ]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['Tweet']).reshape(-1, 1), np.array(df['Label']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['Tweet', 'Label']);

In [ ]:
X = train_os['Tweet'].values
y = train_os['Label'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)

In [ ]:
new_index = pd.RangeIndex(len(X_train))
X_train = pd.Series(X_train, index=new_index)
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
# test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [ ]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()

In [ ]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(bert_model, max_len=MAX_LEN):

    ##params###
    import tensorflow.keras.backend as K
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.CategoricalCrossentropy()
   # accuracy = tf.keras.metrics.CategoricalAccuracy()
    class F1Score(tf.keras.metrics.Metric):
      def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

      def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

      def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

      def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))
    METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      F1Score(name='f1_score')]


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    embeddings = bert_model([input_ids,attention_masks])[1]

    output = tf.keras.layers.Dense(4, activation="softmax")(embeddings)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

    model.compile(opt, loss=loss, metrics=METRICS)


    return model

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=20, batch_size=5)

Epoch 1/20
1322/1322 [==============================] - ETA: 0s - loss: 0.3614 - accuracy: 0.8702 - precision: 0.9330 - recall: 0.8111 - f1_score: 0.8678

/usr/local/lib/python3.9/dist-packages/keras/engine/training.py:2416: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


1322/1322 [==============================] - 279s 173ms/step - loss: 0.3614 - accuracy: 0.8702 - precision: 0.9330 - recall: 0.8111 - f1_score: 0.8678 - val_loss: 0.2316 - val_accuracy: 0.9122 - val_precision: 0.9487 - val_recall: 0.8838 - val_f1_score: 0.9151
Epoch 2/20
1322/1322 [==============================] - 217s 164ms/step - loss: 0.1385 - accuracy: 0.9546 - precision: 0.9659 - recall: 0.9463 - f1_score: 0.9560 - val_loss: 0.1590 - val_accuracy: 0.9516 - val_precision: 0.9594 - val_recall: 0.9437 - val_f1_score: 0.9515
Epoch 3/20
1322/1322 [==============================] - 215s 162ms/step - loss: 0.0671 - accuracy: 0.9800 - precision: 0.9822 - recall: 0.9782 - f1_score: 0.9802 - val_loss: 0.1554 - val_accuracy: 0.9564 - val_precision: 0.9581 - val_recall: 0.9540 - val_f1_score: 0.9560
Epoch 4/20
1322/1322 [==============================] - 217s 164ms/step - loss: 0.0368 - accuracy: 0.9899 - precision: 0.9905 - recall: 0.9891 - f1_score: 0.9898 - val_loss: 0.1777 - val_accuracy